In [217]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch.nn.functional as F

In [218]:
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

In [221]:
data=pd.read_csv("diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [222]:
data.isnull().sum


<bound method DataFrame.sum of      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin    BMI  \
0          False    False          False          False    False  False   
1          False    False          False          False    False  False   
2          False    False          False          False    False  False   
3          False    False          False          False    False  False   
4          False    False          False          False    False  False   
..           ...      ...            ...            ...      ...    ...   
763        False    False          False          False    False  False   
764        False    False          False          False    False  False   
765        False    False          False          False    False  False   
766        False    False          False          False    False  False   
767        False    False          False          False    False  False   

     DiabetesPedigreeFunction    Age  Outcome  
0                   

In [225]:
data["Outcome"].unique()

array([1, 0], dtype=int64)

In [226]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [227]:
x=data.drop("Outcome",axis=1)

In [228]:
sc=StandardScaler()
x=sc.fit_transform(x)

In [229]:
y=data["Outcome"].values

In [230]:
x=x.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [231]:
x=torch.from_numpy(x)
y=torch.from_numpy(y).view(-1,1)

In [232]:
x.shape,y.shape

(torch.Size([768, 8]), torch.Size([768, 1]))

In [233]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [234]:
X_train.shape

torch.Size([537, 8])

In [251]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN,self).__init__()
        self.l1 = torch.nn.Linear(8,6)
        self.l2 = torch.nn.Linear(6,4)
        self.l3 = torch.nn.Linear(4,2)
        self.l4 = torch.nn.Linear(2,1)
        
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        out3 = self.sigmoid(self.l3(out2))
        y_pred = self.sigmoid(self.l4(out3))
        return y_pred
    
model = ANN()

In [252]:
model=ANN()

In [253]:
loss_fn=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [254]:
train=torch.utils.data.TensorDataset(X_train,y_train)
test=torch.utils.data.TensorDataset(X_test,y_test)

In [255]:
train_loader=torch.utils.data.DataLoader(train,batch_size=3,shuffle=True)
test_loader=torch.utils.data.DataLoader(test,batch_size=3,shuffle=True)

In [256]:
epochs = 100

# Some lists to keep track of loss and accuracy during each epoch
epoch_list = []
train_loss_list = []
val_loss_list = []
train_acc_list = []
val_acc_list = []

# Set the training mode ON -> Activate Dropout Layers
model.train() # prepare model for training

for epoch in range(epochs):
    # monitor training loss
    train_loss = 0.0
    val_loss = 0.0
    
    ###################
    # train the model #
    ###################
    
    # Calculate Accuracy         
    correct = 0
    total = 0
    for data,target in train_loader:
#         data=data.view(data.size(0),-1).float()
        data = Variable(data).float()
        target = Variable(target).type(torch.FloatTensor)
        #print("Target = ",target[0].item())
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        predicted = (torch.round(output.data[0]))
        # Total number of labels
        total += len(target)
        # Total correct predictions
        correct += (predicted == target).sum()

        # calculate the loss
        loss = loss_fn(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)

    # calculate average training loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    
    # Avg Accuracy
    accuracy = 100 * correct / float(total)
    # Put them in their list
    train_acc_list.append(accuracy)
    train_loss_list.append(train_loss)
    print('Epoch: {} \tTraining Loss: {:.4f}\t Acc: {:.2f}%'.format(
        epoch+1, 
        train_loss,
        accuracy
        ))
    # Move to next epoch
    epoch_list.append(epoch + 1)

Epoch: 1 	Training Loss: 0.2336	 Acc: 64.00%
Epoch: 2 	Training Loss: 0.2315	 Acc: 64.00%
Epoch: 3 	Training Loss: 0.2303	 Acc: 64.00%
Epoch: 4 	Training Loss: 0.2297	 Acc: 64.00%
Epoch: 5 	Training Loss: 0.2293	 Acc: 64.00%
Epoch: 6 	Training Loss: 0.2291	 Acc: 64.00%
Epoch: 7 	Training Loss: 0.2290	 Acc: 64.00%
Epoch: 8 	Training Loss: 0.2289	 Acc: 64.00%
Epoch: 9 	Training Loss: 0.2289	 Acc: 64.00%
Epoch: 10 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 11 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 12 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 13 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 14 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 15 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 16 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 17 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 18 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 19 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 20 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 21 	Training Loss: 0.2288	 Acc: 64.00%
Epoch: 22 	Training Loss: 0.2288	 Acc: 64.0